# Alle noodzakelijke informatie over je nieuwe API-aansluiting

Hieronder vind je alle informatie die je nodig hebt om gebruik te kunnen maken van mistral medium 3 met je API Key. Belangerijkste punten op een rij:
- Je API Key dat je eerder hebt gekregen blijft onverandert.
- Je API verloopdatum blijft ook onverandert.
- Vanaf nu tot 17/11/2025: je API geeft je toegang tot 2 modellen ipv 1; llama 3 70B en Mistral Medium 3.
- Vanaf 17/11/2025: je API key geeft je alleen maar toegang tot Mistral Medium 3.
- Je hoeft alleen maar 1 variable te wisselen om tussen models te wisselen en dat is je deployment naam.
- In tegenstelling tot Llama is de chat template van Mistral restrictiever; alleen de rollen 'user', 'assistant', 'system' en 'tool' worden ondersteund. De chat template is [hier](https://github.com/vllm-project/vllm/blob/main/examples/tool_chat_template_mistral3.jinja) te raadplegen. Voor de meeste gebruikers en voor meeste use-cases is de 'user'-rol voldoende.

Hieronder zal worden uitgelegd hoe je met mistral aan de salg kan gaan.

In [36]:
import os
import dotenv

dotenv.load_dotenv(dotenv_path=".env")

api_key = os.environ.get("VLAM_API_KEY")
base_url = os.environ.get("VLAM_BASE_URL", "https://api.demo.vlam.ai/v2.1/projects/poc/openai-compatible/v1")
model_id = os.environ.get("VLAM_MODEL_ID", "ubiops-deployment/bzk-bsw-chat//chat-model")

Wat heb je nodig om te starten:
- De notebook die je van ons hebt ontvangen via email
- Je API Key / UbiOps Token: ziet er zo uit "Token 8736rvdf765ds98276edvfs7965dsf8762134". Deze heb je via SecureFileTransfer of via Teams gekregen.
- je deployment_name: bestaat uit drie delen en ziet er zo uit ==> min-afd-chat (ministerie - afdeling - chat).


In [37]:
# ubiops_token = "Token a7da195f09c3743db7fb5c5f30c9e42cb894f7f0" # verander dit mer je eigen Token
ubiops_token = f"Token {api_key}" # verander dit mer je eigen Token

##################################################################################################
## plaats hieronder je deployment_name en vervang `chat` met `mistralmedium-flexibel`.
## Voorbeelden:
## bzk-rvb-chat    ==> bzk-rvb-mistralmedium-flexibel
## logius-dig-chat ==> logius-kcc-mistralmedium-flexibel
## vws-hos-chat    ==> vws-hos-mistralmedium-flexibel
## ienw-pov-chat   ==> ienw-pov-mistralmedium-flexibel
## ssc-test-chat ==> ssc-test-mistralmedium-flexibel

# deployment_name = "ssc-test-mistralmedium-flexibel"
deployment_name = "bzk-bsw-chat"
# deployment_name = "ssc-test-chat"
##################################################################################################


## blijft onverandert
project_name = "poc"
version_name = "" # Leave blank to send request to the default version.
model_name = "chat-model"
base_url = f"https://api.demo.vlam.ai/v2.1/projects/{project_name}/openai-compatible/v1"
model_id=f"ubiops-deployment/{deployment_name}/{version_name}/{model_name}"

# **Vanaf hier is dit precies dezelfde als de notebook dat je van ons ooit hebt gekregen.**

De belangrijkste informatie die je nodig hebt is je **API Base URL** en je **Model-ID**. Dan kan je het in principe gebruiken op de manieren die OpenAI aanbied voor hun chatmodel. Bijvoorbeeld als je het model wilt intergreren met een user interface.

In [27]:
print("API Base URL:\t" + base_url)
print("Model-ID: \t" + model_id)

API Base URL:	https://api.demo.vlam.ai/v2.1/projects/poc/openai-compatible/v1
Model-ID: 	ubiops-deployment/bzk-bsw-chat//chat-model


# Voorbeeld Python OpenAI interface

De OpenAI API biedt een eenvoudige interface voor geavanceerde AI-modellen voor tekstgeneratie en natuurlijke taalverwerking. In dit voorbeeld wordt tekst gegenereerd op basis van een prompt, zoals je dat ook zou doen met ChatGPT.

In [28]:
# %pip install --upgrade openai

In [29]:
from openai import OpenAI

# Generate the OpenAI client
client = OpenAI(
    api_key=ubiops_token[6:] if ubiops_token.startswith("Token ") else ubiops_token,
    base_url=base_url
)
# Send a (streaming) request to the deployment
completion = client.chat.completions.create(
    model=model_id,
    messages=[
        {
            "role": "user",
            "content": "Are you up and running?"
        }
    ],
    # temperature = 0.1,
    stream=False
)
print(completion.choices[0].message.content)

I'm all set and ready to go. How can I assist you today?


In de API-response kun je naast het gegenereerde antwoord van het model ook interessante metadata vinden, zoals:

In [30]:
print("Model:", completion.model)  # Welk model het antwoord genereerde
print("ID:", completion.id)        # Unieke ID van deze API-call (voor logging/debugging)
print("Prompt tokens:", completion.usage.prompt_tokens)        # Hoeveel tokens je input kostte
print("Completion tokens:", completion.usage.completion_tokens)  # Hoeveel tokens het antwoord kostte
print("Total tokens:", completion.usage.total_tokens)          # Totale kosten (input + output)

Model: llama-3.3-70b-instruct-awq
ID: chatcmpl-270ec07422784ab7a876559c6c7e0652
Prompt tokens: 41
Completion tokens: 17
Total tokens: 58


# Voorbeeld met Curl

Je kan het model ook gewoon rechtstreek benaderen via een HTTP verzoek met bijvoorbeeld Curl.  

In [31]:
################################ Hieronder moet je ook je Token invullen. En je deployment naam vervangen ###################################

In [32]:
!curl -X POST "https://api.demo.vlam.ai/v2.1/projects/poc/openai-compatible/v1/chat/completions" \
  -H "Authorization: Token ..." \
  -H "Content-Type: application/json" \
  -d '{"model": "ubiops-deployment/min-test-chat//chat-model", "messages": [{"role": "user", "content": "Are you up and running?"}], "stream": false}'

{"code":"404","message":"Deployment was not found, please make sure it exists","StatusCode":404}

In [51]:
!source .env
# !echo "$VLAM_API_KEY"

In [50]:
!curl -X POST "https://api.demo.vlam.ai/v2.1/projects/poc/openai-compatible/v1/chat/completions" \
  -H "Authorization: Token $VLAM_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{"model": "ubiops-deployment/bzk-bsw-chat//chat-model", "messages": [{"role": "user", "content": "Are you up and running?"}], "stream": false}' | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   769  100   627  100   142    335     76  0:00:01  0:00:01 --:--:--   411
{
  "id": "chatcmpl-7d88012b92dc4e20a5602b5a8851323a",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "I'm all set and ready to go. How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning_content": null
      },
      "stop_reason": null
    }
  ],
  "created": 1761124958,
  "model": "llama-3.3-70b-instruct-awq",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 41,
    "total_tokens": 58,
    "completion_tokens_details"

In [33]:
!curl -X POST "https://api.demo.vlam.ai/v2.1/projects/poc/openai-compatible/v1/chat/completions" \
  -H "Authorization: Token ..." \
  -H "Content-Type: application/json" \
  -d '{"model": "ubiops-deployment/min-test-mistralmedium-flexibel//chat-model", "messages": [{"role": "user", "content": "Are you up and running?"}], "stream": false}'

{"code":"404","message":"Deployment was not found, please make sure it exists","StatusCode":404}

In [54]:
!curl -X POST "https://api.demo.vlam.ai/v2.1/projects/poc/openai-compatible/v1/chat/completions" \
  -H "Authorization: Token $VLAM_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{"model": "ubiops-deployment/bzk-bsw-mistralmedium-flexibel//chat-model", "messages": [{"role": "user", "content": "Are you up and running?"}], "stream": false}' | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1000  100   840  100   160    562    107  0:00:01  0:00:01 --:--:--   669
{
  "id": "chatcmpl-a307454709494ae9bf4bb70f4f320194",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes! I'm fully operational and ready to help with anything you need—whether it's answering questions, brainstorming ideas, solving problems, or just having a chat. 😊 What can I do for you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning_content": null
      },
      "stop_reason": null,
      "token_ids": null
    }
  ],
  "created": 1761125061,
  "model": "mistral-medium-2508",
  "object": "chat.completion",
  "service_tier": nu